## Practical Exercises

In [32]:
import imageio
import matplotlib.pyplot as plt
import elastix
import numpy as np
import os
import SimpleITK as sitk


In [33]:

# IMPORTANT: these paths may differ on your system, depending on where
# Elastix has been installed. Please set accordingly.
ELASTIX_PATH = os.path.join(r'C:/Users/20212059/Documents/CapitaSelecta/elastix-5.0.0-win64/elastix.exe')
TRANSFORMIX_PATH = os.path.join(r'C:/Users/20212059/Documents/CapitaSelecta/elastix-5.0.0-win64/transformix.exe')


In [34]:

if not os.path.exists(ELASTIX_PATH):
    raise IOError('Elastix cannot be found, please set the correct ELASTIX_PATH.')
if not os.path.exists(TRANSFORMIX_PATH):
    raise IOError('Transformix cannot be found, please set the correct TRANSFORMIX_PATH.')

if os.path.exists('../ImagesforPractical/MR/results') is False:
    os.mkdir('../ImagesforPractical/MR/results')


In [35]:
# path = "../ImagesforPractical/MR/patient1.jpg"
# data = imageio.imread(path)
# fig, ax = plt.subplots(1,3, figsize=(16,10))
# ax[0].imshow(data[:,:,0], cmap='gray')
# ax[1].imshow(data[:,:,1], cmap='gray')
# ax[2].imshow(data[:,:,2], cmap='gray')



In [36]:
# Define the paths to the two images you want to register
fixed_image_path = os.path.join('../ImagesforPractical/MR/patient1.jpg')
moving_image_path = os.path.join('../ImagesforPractical/MR/patient2.jpg')


el = elastix.ElastixInterface(elastix_path=ELASTIX_PATH)
el.register(
    fixed_image=fixed_image_path,
    moving_image=moving_image_path,
    parameters=[os.path.join('../ImagesforPractical/MR/parameters_bspline_multires_MR.txt')],
    output_dir='../ImagesforPractical/MR/results')


C:/Users/20212059/Documents/CapitaSelecta/elastix-5.0.0-win64/elastix.exe -f ../ImagesforPractical/MR/patient1.jpg -m ../ImagesforPractical/MR/patient2.jpg -p ../ImagesforPractical/MR/parameters_bspline_multires_MR.txt -out ../ImagesforPractical/MR/results
